In [191]:
import sqlite3
import re

In [192]:
con = sqlite3.connect("db/main.db")
cur = con.cursor()

In [193]:
cur.execute("""CREATE TABLE IF NOT EXISTS "Log_table" (
	"id"	INTEGER,
	"date"	TEXT,
	"text"	TEXT,
	"label"	TEXT,
	"info"	TEXT,
	PRIMARY KEY("id")
);""")
con.commit()

In [194]:
cur.execute("""CREATE TABLE IF NOT EXISTS "Sample_table" (
	"id"	INTEGER,
	"text_en"	TEXT,
	"text_ru"	TEXT DEFAULT NULL,
	"label"	TEXT,
	"processed"	INTEGER DEFAULT 0,
	PRIMARY KEY("id")
);""")
con.commit()

for i in range(10):
    con_old = sqlite3.connect(f"homework/data/db/mydatabase_{i}.db")
    cur_old = con_old.cursor()

    res = cur_old.execute("Select * from emotions").fetchall()
    for item in res:
        cur.execute("INSERT INTO \"Sample_table\" (text_en, text_ru, label, processed) VALUES (?, ?, ?, ?)",
                              (item[2], item[3], item[1], 0))

    con_old.commit()
    con_old.close()
    con.commit()

In [195]:
cur.execute("""CREATE TABLE IF NOT EXISTS "Usage_table" (
	"id"	INTEGER,
	"word"	TEXT NOT NULL,
	"language"	TEXT NOT NULL,
	"label"	TEXT NOT NULL,
	"usage"	INTEGER NOT NULL DEFAULT 0,
	PRIMARY KEY("id")
);""")
con.commit()

# cur.execute("""Delete from "Usage_table";""")
# con.commit()


# # res = cur.execute("""
# # Select * from "Sample_table" where "text_ru" is not NULL limit 5;
# # """).fetchall()

# res = cur.execute("""
# Select * from "Sample_table" where "text_ru" is not NULL;
# """).fetchall()

# for row in res:
# 	# label
# 	label = row[3] 
# 	for word in row[2].split():
# 		# rus text
# 		# to lower and delete comma's and dots
# 		new_word = re.sub(r"[\"\'.]", r"", word.lower())
# 		print(new_word, end=" ")
# 		count = cur.execute(f"""select usage from "Usage_table" where word is "{new_word}";""").fetchall()
# 		if count == []:
# 			count = 0
# 			cur.execute("insert into \"Usage_table\" (word, language, label, usage) values (?, ?, ?, ?)", (new_word, "ru", label, str(count)))
# 		else:
# 			count = int(count[0][0])
# 			cur.execute(f"update \"Usage_table\" set usage = {count + 1} where word = \"{new_word}\" and label = \"{label}\";")
# 		print(new_word, count)
# 	con.commit()



In [196]:
con.close()